## Import Libraries

In [ ]:
## Common Libraries
import os
import threading
import time
import shutil
import hashlib
import time
import codecs
import shutil
import tarfile
import subprocess
from pathlib import Path
from datetime import datetime
## Model Libraries
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
print(f"Current CUDA device: {torch.cuda.current_device()}")
print(f"CUDA device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

### Install Libraries

In [ ]:
rot_47 = lambda encoded_text: "".join(
    [
        (
            chr(
                (ord(c) - (ord("a") if c.islower() else ord("A")) - 47) % 26
                + (ord("a") if c.islower() else ord("A"))
            )
            if c.isalpha()
            else c
        )
        for c in encoded_text
    ]
)

E = Exception
B = print


def vidal_setup(ForceIn):
    L = "Kikpm.ovm.bu"
    K = "/content/"
    C = ForceIn

    def F():
        print("Installing pip packages...")
        subprocess.check_call(["pip", "install", "-r", "requirements.txt", "--quiet"])

    A = K + rot_47(L)
    G = K + rot_47(L)
    D = "/"
    if not os.path.exists(A):
        M = os.path.dirname(A)
        os.makedirs(M, exist_ok=True)
        print("No cached install found..")
        try:
            N = rot_47(
                codecs.decode(
                    "pbbxa://pcooqvonikm.kw/QIPqaxivw/Ixxtqw/zmawtdm/uiqv/Kwtij/Xvxcz.biz.oh",
                    "rot_13",
                )
            )
            subprocess.run(["wget", "-O", A, N])
            print("Download completed successfully!")
        except E as H:
            print(str(H))
            if os.path.exists(A):
                os.remove(A)
    if Path(A).exists():
        with tarfile.open(G, "r:gz") as I:
            for J in I.getmembers():
                O = os.path.join(D, J.name)
                try:
                    I.extract(J, D)
                except E as H:
                    print("Failed to extract a file")
                    C = True
            print(f"Extraction of {G} to {D} completed.")
        if os.path.exists(A):
            os.remove(A)
        if C:
            F()
            C = False
    else:
        F()


vidal_setup(False)
print("Finished installing requirements!")

## Parmeters Setting

In [ ]:
model_name = "Aivle_HanKY"  # @param {type:"string"}
dataset_path = "./train_han"  # @param {type:"string"}

sample_rate = "48k"  # @param ["32k", "40k", "48k"] {allow-input: false}
sr = int(sample_rate.rstrip("k")) * 1000
cpu_cores = 2 # @param {type:"slider", min:1, max:2, step:1}

rvc_version = "v2"  # @param ["v2", "v1"] {allow-input: false}
f0_method = "rmvpe"  # @param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe"] {allow-input: false}
pitch_guidance = True  # @param{type:"boolean"}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}

LOGS_FOLDER = "./logs/"
WEIGHTS_FOLDER = os.path.join(LOGS_FOLDER, model_name)
RVC_BK_PATH = "./RVC_Backup"

total_epoch = 200  # @param {type:"integer"}
batch_size = 22  # @param {type:"slider", min:1, max:25, step:0}
gpu = 0
sr = int(sample_rate.rstrip("k")) * 1000
pitch_guidance = True  # @param{type:"boolean"}
auto_backups = True  # @param{type:"boolean"}
pretrained = True  # @param{type:"boolean"}
sync_graph = False  # @param{type:"boolean"}
cache_data_in_gpu = True  # @param{type:"boolean"}
tensorboard = False  # @param{type:"boolean"}
# @markdown ### ➡️ Choose how many epochs your model will be stored
save_every_epoch = 20  # @param {type:"slider", min:1, max:100, step:0}
save_only_latest = False  # @param{type:"boolean"}
save_every_weights = True  # @param{type:"boolean"}
overtraining_detector = False  # @param{type:"boolean"}
overtraining_threshold = 50  # @param {type:"slider", min:1, max:100, step:0}

# Optional
# In case you select custom pretrained, you will have to download the pretraineds and enter the path of the pretraineds.
custom_pretrained = False  # @param{type:"boolean"}
g_pretrained_path = "./rvc/models/pretraineds/pretrained_v2/G48k.pth"  # @param {type:"string"}
d_pretrained_path = "./rvc/models/pretraineds/pretrained_v2/D48k.pth"  # @param {type:"string"}

if "pretrained" not in globals():
    pretrained = True

if "custom_pretrained" not in globals():
    custom_pretrained = False

if "g_pretrained_path" not in globals():
    g_pretrained_path = "Custom Path"

if "d_pretrained_path" not in globals():
    d_pretrained_path = "Custom Path"

model_epoch = 200  # @param {type:"integer"}
save_big_file = False  # @param {type:"boolean"}
model_path = os.path.join("./models", model_name)


## Pre-Processing

In [ ]:
!python core.py prerequisites
!python core.py preprocess --model_name "{model_name}" --dataset_path "{dataset_path}" --sampling_rate "{sr}" --cpu_cores "{cpu_cores}"
!python core.py extract --model_name "{model_name}" --rvc_version "{rvc_version}" --f0_method "{f0_method}" --pitch_guidance "{pitch_guidance}" --hop_length "{hop_length}" --sampling_rate "{sr}" --cpu_cores "{cpu_cores}"

## RVC Training

In [ ]:
def import_rvc_backup():
    print("Importing RVC backup...")  # RVC 백업을 가져오는 작업 시작을 알림
    weights_exist = False  # 가중치 파일 존재 여부를 나타내는 플래그 초기화
    for root, dirs, files in os.walk(RVC_BK_PATH):  # RVC_BK_PATH 디렉토리를 순회하며 파일 목록을 탐색
        for filename in files:  # 각 파일에 대해 반복
            filepath = os.path.join(root, filename)  # 파일의 전체 경로 생성
            if os.path.isfile(filepath) and not filepath.startswith(
                os.path.join(RVC_BK_PATH, "weights")
            ):  # 파일이 존재하고 경로가 RVC_BK_PATH/weights로 시작하지 않는 경우
                backup_filepath = os.path.join(
                    LOGS_FOLDER, os.path.relpath(filepath, RVC_BK_PATH)
                )  # 백업 파일의 경로 생성
                backup_folderpath = os.path.dirname(backup_filepath)  # 백업 폴더 경로 생성
                if not os.path.exists(backup_folderpath):  # 백업 폴더가 존재하지 않는 경우
                    os.makedirs(backup_folderpath)  # 백업 폴더 생성
                    print(f"Created backup folder: {backup_folderpath}", flush=True)  # 폴더 생성 알림 출력
                shutil.copy2(filepath, backup_filepath)  # 파일을 메타데이터와 함께 복사
                print(f"Imported file from RVC backup: {filename}")  # 파일 복사 완료 알림 출력
            elif filepath.startswith(
                os.path.join(RVC_BK_PATH, "weights")
            ) and filename.endswith(".pth"):  # 파일 경로가 RVC_BK_PATH/weights로 시작하고 .pth 확장자인 경우
                weights_exist = True  # 가중치 파일 존재 여부 플래그 설정
                weights_filepath = os.path.join(
                    WEIGHTS_FOLDER,
                    os.path.relpath(
                        filepath, os.path.join(RVC_BK_PATH, "weights")
                    ),
                )  # 가중치 파일의 경로 생성
                weights_folderpath = os.path.dirname(weights_filepath)  # 가중치 폴더 경로 생성
                if not os.path.exists(weights_folderpath):  # 가중치 폴더가 존재하지 않는 경우
                    os.makedirs(weights_folderpath)  # 가중치 폴더 생성
                    print(f"Created weights folder: {weights_folderpath}", flush=True)  # 폴더 생성 알림 출력
                shutil.copy2(filepath, weights_filepath)  # 파일을 메타데이터와 함께 복사
                print(f"Imported file from weights: {filename}")  # 파일 복사 완료 알림 출력
    if weights_exist:  # 가중치 파일이 존재하는 경우
        print("Copied weights from RVC backup to local weights folder.")  # 가중치 파일 복사 완료 알림 출력
    else:  # 가중치 파일이 존재하지 않는 경우
        print("No weights found in RVC backup.")  # 가중치 파일 없음 알림 출력
    print("RVC backup import completed.")  # RVC 백업 가져오기 완료 알림 출력

# 파일의 MD5 해시 값을 계산하는 함수 정의
def get_md5_hash(file_path):
    hash_md5 = hashlib.md5()  # MD5 해시 객체 생성
    with open(file_path, "rb") as f:  # 파일을 바이너리 읽기 모드로 엶
        for chunk in iter(lambda: f.read(4096), b""):  # 파일을 4096 바이트씩 읽어옴
            hash_md5.update(chunk)  # 읽어온 청크를 MD5 해시 객체에 업데이트
    return hash_md5.hexdigest()  # 최종 해시 값을 16진수 문자열로 반환

# 가중치 폴더를 RVC 백업 폴더로 복사하는 함수 정의
def copy_weights():
    destination_folder = os.path.join(RVC_BK_PATH, "weights")  # 목적지 폴더 경로 생성
    try:  # 예외 처리를 위한 try 블록 시작
        if not os.path.exists(destination_folder):  # 목적지 폴더가 존재하지 않는 경우
            os.makedirs(destination_folder)  # 목적지 폴더 생성

        num_copied = 0  # 복사된 파일 개수 초기화
        for filename in os.listdir(WEIGHTS_FOLDER):  # WEIGHTS_FOLDER 내 파일 목록 반복
            if filename.endswith(".pth"):  # 파일명이 .pth로 끝나는 경우
                source_file = os.path.join(WEIGHTS_FOLDER, filename)  # 소스 파일 경로 생성
                destination_file = os.path.join(destination_folder, filename)  # 목적지 파일 경로 생성
                if not os.path.exists(destination_file):  # 목적지 파일이 존재하지 않는 경우
                    shutil.copy2(source_file, destination_file)  # 소스 파일을 목적지 파일로 복사 (메타데이터 포함)
                    num_copied += 1  # 복사된 파일 개수 증가
                    print(f"Copied {filename} to RVC Backup!")  # 복사 완료 메시지 출력

        if num_copied == 0:  # 복사된 파일이 없는 경우
            print("No new finished models found for copying.")  # 복사할 새로운 모델이 없다는 메시지 출력
        else:  # 복사된 파일이 있는 경우
            print(f"Finished copying {num_copied} files to RVC Backup!")  # 복사 완료된 파일 개수 출력

    except Exception as e:  # 예외가 발생한 경우
        print(f"An error occurred while copying weights: {str(e)}")  # 예외 메시지 출력

def backup_files():  # 파일을 백업하는 함수 정의
    print("\nStarting backup loop...")  # 백업 루프 시작 알림 출력
    last_backup_timestamps_path = os.path.join(
        LOGS_FOLDER, "last_backup_timestamps.txt"
    )  # 마지막 백업 타임스탬프 파일 경로 생성
    fully_updated = False  # 모든 파일이 업데이트 되었는지 여부를 나타내는 플래그 초기화

    while True:  # 무한 루프 시작
        try:  # 예외 처리를 위한 try 블록 시작
            updated = False  # 업데이트 여부를 나타내는 플래그 초기화
            last_backup_timestamps = {}  # 마지막 백업 타임스탬프를 저장할 딕셔너리 초기화

            try:  # 파일 읽기를 위한 예외 처리 블록
                with open(last_backup_timestamps_path, "r") as f:  # 타임스탬프 파일 읽기
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)  # 파일 내용을 딕셔너리로 변환
            except FileNotFoundError:  # 파일이 존재하지 않는 경우 예외 처리
                pass  # 예외 발생 시 아무것도 하지 않음

            for root, dirs, files in os.walk(LOGS_FOLDER):  # LOGS_FOLDER 내 파일 및 디렉토리 탐색
                if "zips" in dirs:  # "zips" 디렉토리 제외
                    dirs.remove("zips")
                if "mute" in dirs:  # "mute" 디렉토리 제외
                    dirs.remove("mute")
                for filename in files:  # 파일 목록 반복
                    if filename != "last_backup_timestamps.txt":  # 타임스탬프 파일 제외
                        filepath = os.path.join(root, filename)  # 파일 경로 생성
                        if os.path.isfile(filepath):  # 파일이 존재하는 경우
                            backup_filepath = os.path.join(
                                RVC_BK_PATH,
                                os.path.relpath(filepath, LOGS_FOLDER),
                            )  # 백업 파일 경로 생성
                            backup_folderpath = os.path.dirname(backup_filepath)  # 백업 폴더 경로 생성
                            if not os.path.exists(backup_folderpath):  # 백업 폴더가 존재하지 않는 경우
                                os.makedirs(backup_folderpath)  # 백업 폴더 생성
                                print(
                                    f"Created backup folder: {backup_folderpath}",
                                    flush=True,
                                )  # 폴더 생성 알림 출력
                            last_backup_timestamp = last_backup_timestamps.get(filepath)  # 마지막 백업 타임스탬프 가져오기
                            current_timestamp = os.path.getmtime(filepath)  # 현재 파일의 수정 타임스탬프 가져오기
                            if (
                                last_backup_timestamp is None
                                or float(last_backup_timestamp) < current_timestamp
                            ):  # 파일이 새로 생성되었거나 업데이트된 경우
                                shutil.copy2(filepath, backup_filepath)  # 파일을 백업 경로로 복사 (메타데이터 포함)
                                last_backup_timestamps[filepath] = str(
                                    current_timestamp
                                )  # 최신 타임스탬프 업데이트
                                if last_backup_timestamp is None:  # 새로운 파일인 경우
                                    print(f"Backed up file: {filename}")  # 백업 완료 메시지 출력
                                else:  # 업데이트된 파일인 경우
                                    print(f"Updating backed up file: {filename}")  # 업데이트 완료 메시지 출력
                                updated = True  # 업데이트 플래그 설정
                                fully_updated = False  # 모든 파일 업데이트 플래그 해제

            for filepath in list(last_backup_timestamps.keys()):  # 타임스탬프 딕셔너리 내 파일 경로 반복
                if not os.path.exists(filepath):  # 파일이 삭제된 경우
                    backup_filepath = os.path.join(
                        RVC_BK_PATH, os.path.relpath(filepath, LOGS_FOLDER)
                    )  # 백업 파일 경로 생성
                    if os.path.exists(backup_filepath):  # 백업 파일이 존재하는 경우
                        os.remove(backup_filepath)  # 백업 파일 삭제
                        print(f"Deleted file: {filepath}")  # 파일 삭제 메시지 출력
                    del last_backup_timestamps[filepath]  # 타임스탬프 딕셔너리에서 파일 경로 삭제
                    updated = True  # 업데이트 플래그 설정
                    fully_updated = False  # 모든 파일 업데이트 플래그 해제

            if not updated and not fully_updated:  # 파일이 업데이트되지 않았고 모든 파일이 업데이트된 경우
                print("Files are up to date.")  # 파일이 최신 상태라는 메시지 출력
                fully_updated = True  # 모든 파일 업데이트 플래그 설정
                sleep_time = 15  # 대기 시간 설정
            else:  # 파일이 업데이트된 경우
                sleep_time = 0.1  # 짧은 대기 시간 설정

            with open(last_backup_timestamps_path, "w") as f:  # 타임스탬프 파일 쓰기 모드로 열기
                for filepath, timestamp in last_backup_timestamps.items():  # 타임스탬프 딕셔너리 내 항목 반복
                    f.write(f"{filepath}:{timestamp}\n")  # 파일에 타임스탬프 기록

            time.sleep(sleep_time)  # 설정된 시간 동안 대기

        except Exception as e:  # 예외가 발생한 경우
            print(f"An error occurred: {str(e)}")  # 예외 메시지 출력

# def start_train():
#     !python core.py train --model_name "{model_name}" --rvc_version "{rvc_version}" --save_every_epoch "{save_every_epoch}" --save_only_latest "{save_only_latest}" --save_every_weights "{save_every_weights}" --total_epoch "{total_epoch}" --sampling_rate "{sr}" --batch_size "{batch_size}" --gpu "{gpu}" --pitch_guidance "{pitch_guidance}" --pretrained "{pretrained}" --custom_pretrained "{custom_pretrained}" --g_pretrained_path "{g_pretrained_path}" --d_pretrained_path "{d_pretrained_path}" --overtraining_detector "{overtraining_detector}" --overtraining_threshold "{overtraining_threshold}" --sync_graph "{sync_graph}" --cache_data_in_gpu "{cache_data_in_gpu}"

# server_thread = threading.Thread(target=start_train)
# server_thread.start()
!python core.py train --model_name "{model_name}" --rvc_version "{rvc_version}" --save_every_epoch "{save_every_epoch}" --save_only_latest "{save_only_latest}" --save_every_weights "{save_every_weights}" --total_epoch "{total_epoch}" --sampling_rate "{sr}" --batch_size "{batch_size}" --gpu "{gpu}" --pitch_guidance "{pitch_guidance}" --pretrained "{pretrained}" --custom_pretrained "{custom_pretrained}" --g_pretrained_path "{g_pretrained_path}" --d_pretrained_path "{d_pretrained_path}" --overtraining_detector "{overtraining_detector}" --overtraining_threshold "{overtraining_threshold}" --sync_graph "{sync_graph}" --cache_data_in_gpu "{cache_data_in_gpu}"

## Generate index file

In [ ]:
!python core.py index --model_name "{model_name}" --rvc_version "{rvc_version}"

## Save RVC Model

In [ ]:
if os.path.exists(model_path):  # model_path 경로가 존재하는지 확인
    shutil.rmtree(model_path)  # 해당 경로 및 그 하위 파일/폴더 삭제
print("Removed zips.")  # 삭제 완료 메시지 출력

os.makedirs(model_path)  # model_path 경로 생성
print("Created zips.")  # 생성 완료 메시지 출력

model_pth = os.path.join(model_path, f"{model_name}.pth")  # 모델 파일 경로 생성
if model_pth not in os.listdir("./weights"):  # 모델 파일이 백업 폴더에 없는지 확인
    print("There is no weight file with that name")  # 해당 이름의 가중치 파일이 없음을 알림

if not save_big_file:  # save_big_file 플래그가 False인 경우
    !cp ./logs/{model_name}/added_*.index ./models/{model_name}/  # 로그 파일을 models 폴더로 복사
    !cp ./logs/{model_name}/total_*.npy ./models/{model_name}/  # 로그 파일을 models 폴더로 복사
    !cp ./logs/{model_name}_200e_*.pth ./models/{model_name}/{model_name}.pth
    # !cp ./weights/{model_name}.pth ./models/{model_name}/{model_name}.pth  # 가중치 파일을 models 폴더로 복사
    # %cd ./models/{model_name}/  # zips 폴더로 이동
    !zip -r ./models/{model_name}/{model_name}.zip ./models/{model_name}  # models 폴더의 파일들을 압축

if save_big_file:  # save_big_file 플래그가 True인 경우
    latest_steps = -1  # 최신 스텝 초기화
    logs_folder = "./logs/" + model_name  # 로그 폴더 경로 생성
    for filename in os.listdir(logs_folder):  # 로그 폴더의 파일 목록 반복
        if filename.startswith("G_") and filename.endswith(".pth"):  # G_로 시작하고 .pth로 끝나는 파일 찾기
            steps = int(filename.split("_")[1].split(".")[0])  # 스텝 수 추출
            if steps > latest_steps:  # 최신 스텝 수 업데이트
                latest_steps = steps

    MODELZIP = model_name + ".zip"  # 모델 압축 파일 이름 생성
    !mkdir -p ./models
    ZIPFILEPATH = os.path.join("./models", MODELZIP)  # 압축 파일 경로 생성
    for filename in os.listdir(logs_folder):  # 로그 폴더의 파일 목록 반복
        if "G_" in filename or "D_" in filename:  # G_ 또는 D_가 파일명에 포함된 경우
            if str(latest_steps) in filename:  # 최신 스텝 수가 파일명에 포함된 경우
                !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}  # 파일을 압축
        else:  # 그 외 파일
            !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}  # 파일을 압축
    for filename in os.listdir("./weights"):  # weights 폴더의 파일 목록 반복
        if model_name in filename:  # 모델 이름이 파일명에 포함된 경우
            !zip -r {ZIPFILEPATH} {os.path.join('./weights/', filename)}  # 파일을 압축

shutil.move(
    f"./models/{model_name}/{model_name}.zip",
    f"./RVC_Backup/{model_name}.zip",
)  # 압축 파일을 백업 폴더로 이동

# shutil.rmtree("/home/chwang/KT/BP/voice_generator/applio_content/zips")  # zips 폴더 삭제